# Multiclass Plant Image Classification Using A Convolutional Neural Network
For our project, we aim to predict the maturity and species of plant seedling images using a convolutional neural network model. To accomplish this, we implement a convolutional neural network model in Tensorflow.

In [0]:
import numpy as np
import pandas as pd

import os
import shutil
import logging
import random

from PIL import Image
from tqdm import tqdm

from google.colab import drive

import tensorflow as tf
from tqdm import trange
from sklearn import preprocessing

#drive.mount('/content/drive',force_remount=True)

# desired image size (64) and directory paths
DIMENSIONS = 64
DRIVE_FOLDER = '/content/drive/My Drive/Colab Notebooks'
MAIN_FOLDER = 'all_seedlings'

## Preparing the dataset
### Resizing, renaming, and splitting the data
Our model will be trained on an image dataset provided by Kaggle to classify 12 types of plant seedlings, captured at different stages of growth. To prepare the data for input into the model, we have to perform a number of tasks:
* Systematically rename all the images to be able to access both the image ID and class (species) label 
  * The seedling dataset comes in the following format:
    
    `all_seedlings`/
        ...
        {speciesLabel}_{imageID}.png
        May Flower_123.png
        ...
* Downsize the images to standard dimensions: 64 x 64 
  * Resizing the image size standardizes the images for input and makes training faster by minimizing image loading and processing time.
* Split the dataset into train/dev/test sets
  * Because we don't have a lot of images and we want the statistics on the dev set to be as representative as possible, we'll take 10% of training data as the dev_set and another 10% as the test_set.
* Load the data into Tensorflow vectors
* Encode the species labels to numerical types


In [0]:
'''
BUILD_DATASET(dims):
 - dims: desired image size

Prepares a dataset of dims x dims-sized images for input into the model by:
 - resizing the images
 - renaming their file paths
 - moving all classes into the same folder
''' 
def build_dataset(dims):

  # get list of individual species folders
  species = os.listdir(os.path.join(DRIVE_FOLDER, 'seedlings'))

  count = 0
  for folder in species:
    
    # get path to the folder
    path = os.path.join(DRIVE_FOLDER, 'seedlings', str(folder))

    # create a list of all images in the folder
    images = os.listdir(path)
    
    # move the images to the same folder ("all_seedlings")
    for img_name in images:

        # original source path to image
        src = os.path.join(path, img_name)
        
        # open image
        image = Image.open(src)
        
        # resize image using bilinear interpolation instead of nearest neighbors
        resized_image = image.resize((DIMENSIONS, DIMENSIONS), Image.BILINEAR)
  
        # change the file name to include class (species name) 
        new_img_name = str(folder) + '_' + img_name
        
        # destination path to image
        dst = os.path.join(DRIVE_FOLDER, MAIN_FOLDER, new_img_name)
        
        resized_image.save(dst)
        
        count += 1
        
    print("Resized", count, "images.")

      
        
build_dataset(DIMENSIONS)

In [2]:
print("There are", len(os.listdir(os.path.join(DRIVE_FOLDER, MAIN_FOLDER))), "total images in the original dataset.")

There are 5539 total images in the original dataset.


In [0]:
'''
GET_SPECIES(filename):
 - filename: The filename of an image, in the format {speciesLabel}_{imageID}.png
Extracts the species class label from an image filename.
'''
def get_species(filename):
  species, img_id = filename.split('_')
  return species

In [0]:
'''
SPLIT_TRAIN_DEV_TEST(data, train_size):
 - data: Original dataset to be split into training, dev, and test sets
 - train_size: Fraction of dataset to be devoted to training data
 Splits a dataset into a training, dev, and test sets given a specified training set size.
 The dev and test sets are equally sized from the remaining data. 
'''
def split_train_dev_test(data, train_size):

  # make sure to always shuffle with a fixed seed so that the split is reproducible
  random.seed(101)
  data.sort()
  random.shuffle(data)

  # split the images in 80% train, 10% dev, 10% test
  pivot = int((1 - train_size) * len(data))
  train_filenames = data[pivot:]
  pivot_half = int(pivot / 2)
  test_filenames = data[0:pivot_half]
  dev_filenames = data[pivot_half:pivot]
  
  split_filenames = {'train': train_filenames,
              'dev': dev_filenames,
              'test': test_filenames}        
              
  for split in ['train', 'dev', 'test']:
    # create directory to hold new folder
    output_dir_split = os.path.join(DRIVE_FOLDER, split)

    if not os.path.exists(output_dir_split):
      os.mkdir(output_dir_split)
    else:
      print("Warning: dir {} already exists".format(output_dir_split))

    print("Saving preprocessed data to {}".format(output_dir_split))
    
    for img in split_filenames[split]:
      src = os.path.join(DRIVE_FOLDER, MAIN_FOLDER, img)
      dest = os.path.join(output_dir_split, img)

      shutil.copyfile(src,dest)
      print('.')

# get all images from the new standard folder
dataset = os.listdir(os.path.join(DRIVE_FOLDER, MAIN_FOLDER))
print("There are", len(dataset), "total images in this dataset.")

t = 0.8
split_train_dev_test(dataset, t)

trainers = os.listdir(os.path.join(DRIVE_FOLDER, "train"))
print("There are", len(trainers), "training images.")

devs = os.listdir(os.path.join(DRIVE_FOLDER, "dev"))
print("There are", len(devs), "development images.")

testers = os.listdir(os.path.join(DRIVE_FOLDER, "test"))
print("There are", len(testers), "test images.")



In [4]:
# get all images from the new standard folder
dataset = os.listdir(os.path.join(DRIVE_FOLDER, MAIN_FOLDER))
print("There are", len(dataset), "total images in this dataset.")

t = 0.8
#split_train_dev_test(dataset, t)

trainers = os.listdir(os.path.join(DRIVE_FOLDER, "train"))
print("There are", len(trainers), "training images.")

devs = os.listdir(os.path.join(DRIVE_FOLDER, "dev"))
print("There are", len(devs), "development images.")

testers = os.listdir(os.path.join(DRIVE_FOLDER, "test"))
print("There are", len(testers), "test images.")

There are 5539 total images in this dataset.
There are 4432 training images.
There are 554 development images.
There are 553 test images.


In [0]:
def set_logger(log_path):
    """Sets the logger to log info in terminal and file `log_path`.
    In general, it is useful to have a logger so that every output to the terminal is saved
    in a permanent file. Here we save it to `model_dir/train.log`.
    Example:
    ```
    logging.info("Starting training...")
    ```
    Args:
        log_path: (string) where to log
    """
    logger = logging.getLogger()
    logger.setLevel(logging.INFO)

    if not logger.handlers:
        # Logging to a file
        file_handler = logging.FileHandler(log_path)
        file_handler.setFormatter(logging.Formatter('%(asctime)s:%(levelname)s: %(message)s'))
        logger.addHandler(file_handler)

        # Logging to console
        stream_handler = logging.StreamHandler()
        stream_handler.setFormatter(logging.Formatter('%(message)s'))
        logger.addHandler(stream_handler)


In [0]:
def _parse_function(filename, label, size):
    """Obtain the image from the filename (for both training and validation).
    The following operations are applied:
        - Decode the image from png format
        - Convert to float and to range [0, 1]
    """
    image_string = tf.read_file(filename)

    image_decoded = tf.image.decode_png(image_string, channels=3)

    # This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image_decoded, tf.float32)

    resized_image = tf.image.resize_images(image, [size, size])

    return resized_image, label
  
def train_preprocess(image, label, use_random_flip):
    """Image preprocessing for training.
    Apply the following operations:
        - Horizontally flip the image with probability 1/2
        - Apply random brightness and saturation
    """
    if use_random_flip:
        image = tf.image.random_flip_left_right(image)

    image = tf.image.random_brightness(image, max_delta=32.0 / 255.0)
    image = tf.image.random_saturation(image, lower=0.5, upper=1.5)

    # Make sure the image is still in [0, 1]
    image = tf.clip_by_value(image, 0.0, 1.0)

    return image, label
  
def input_fn(is_training, filenames, labels, params):
    """
    The filenames have format "{speciesLabel}_{idNumber}.png".
    For instance: "Scentless Mayweed_29.png".
    Args:
        is_training: (bool) whether to use the train or test pipeline.
                     At training, we shuffle the data and have multiple epochs
        filenames: (list) filenames of the images, as ["data_dir/{label}_IMG_{id}.jpg"...]
        labels: (list) corresponding list of species labels
        params: (Params) contains hyperparameters of the model (ex: `params.num_epochs`)
    """
    
    num_samples = len(filenames)
    assert len(filenames) == len(labels), "Filenames and labels should have same length"

    # Create a Dataset serving batches of images and labels
    # We don't repeat for multiple epochs because we always train and evaluate for one epoch
    parse_fn = lambda f, l: _parse_function(f, l, params["image_size"])
    train_fn = lambda f, l: train_preprocess(f, l, params["use_random_flip"])

    if is_training:
        dataset = (tf.data.Dataset.from_tensor_slices((tf.constant(filenames), tf.constant(labels)))
            .shuffle(num_samples)  # whole dataset into the buffer ensures good shuffling
            .map(parse_fn, num_parallel_calls=params["num_parallel_calls"])
            .map(train_fn, num_parallel_calls=params["num_parallel_calls"])
            .batch(params["batch_size"])
            .prefetch(1)  # make sure you always have one batch ready to serve
        )
    else:
        dataset = (tf.data.Dataset.from_tensor_slices((tf.constant(filenames), tf.constant(labels)))
            .map(parse_fn)
            .batch(params["batch_size"])
            .prefetch(1)  # make sure you always have one batch ready to serve
        )

    # Create reinitializable iterator from dataset
    iterator = dataset.make_initializable_iterator()
    images, labels = iterator.get_next()
    iterator_init_op = iterator.initializer

    inputs = {'images': images, 'labels': labels, 'iterator_init_op': iterator_init_op}
    return inputs

In [0]:
def build_model(is_training, inputs, params):
    """Compute logits of the model (output distribution)
    Args:
        is_training: (bool) whether we are training or not
        inputs: (dict) contains the inputs of the graph (features, labels...)
                this can be `tf.placeholder` or outputs of `tf.data`
        params: (Params) hyperparameters
    Returns:
        output: (tf.Tensor) output of the model
    """
    images = inputs['images']

    assert images.get_shape().as_list() == [None, params["image_size"], params["image_size"], 3]

    out = images
    # Define the number of channels of each convolution
    # For each block, we do: 3x3 conv -> batch norm -> relu -> 2x2 maxpool
    num_channels = params["num_channels"]
    bn_momentum = params["bn_momentum"]
    channels = [num_channels, num_channels * 2, num_channels * 4, num_channels * 8]
    for i, c in enumerate(channels):
        with tf.variable_scope('block_{}'.format(i+1)):
            out = tf.layers.conv2d(out, c, 3, padding='same')
            if params["use_batch_norm"]:
                out = tf.layers.batch_normalization(out, momentum=bn_momentum, training=is_training)
            out = tf.nn.relu(out)
            out = tf.layers.max_pooling2d(out, 2, 2)

    assert out.get_shape().as_list() == [None, 4, 4, num_channels * 8]

    out = tf.reshape(out, [-1, 4 * 4 * num_channels * 8])
    with tf.variable_scope('fc_1'):
        out = tf.layers.dense(out, num_channels * 8)
        if params["use_batch_norm"]:
            out = tf.layers.batch_normalization(out, momentum=bn_momentum, training=is_training)
        out = tf.nn.relu(out)
    with tf.variable_scope('fc_2'):
        logits = tf.layers.dense(out, params["num_labels"])
        
    return logits


def model_fn(mode, inputs, params, reuse=False):
    """Model function defining the graph operations.
    Args:
        mode: (string) can be 'train' or 'eval'
        inputs: (dict) contains the inputs of the graph (features, labels...)
                this can be `tf.placeholder` or outputs of `tf.data`
        params: (Params) contains hyperparameters of the model (ex: `params.learning_rate`)
        reuse: (bool) whether to reuse the weights
    Returns:
        model_spec: (dict) contains the graph operations or nodes needed for training / evaluation
    """
    is_training = (mode == 'train')
    labels = inputs['labels']
    labels = tf.cast(labels, tf.int64)

    # -----------------------------------------------------------
    # MODEL: define the layers of the model
    with tf.variable_scope('model', reuse=reuse):
        # Compute the output distribution of the model and the predictions
        logits = build_model(is_training, inputs, params)
        predictions = tf.argmax(logits, 1)

    # Define loss and accuracy
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(labels, predictions), tf.float32))

    # Define training step that minimizes the loss with the Adam optimizer
    if is_training:
        optimizer = tf.train.AdamOptimizer(params["learning_rate"])
        global_step = tf.train.get_or_create_global_step()
        if params["use_batch_norm"]:
            # Add a dependency to update the moving mean and variance for batch normalization
            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                train_op = optimizer.minimize(loss, global_step=global_step)
        else:
            train_op = optimizer.minimize(loss, global_step=global_step)


    # -----------------------------------------------------------
    # METRICS AND SUMMARIES
    # Metrics for evaluation using tf.metrics (average over whole dataset)
    with tf.variable_scope("metrics"):
        metrics = {
            'accuracy': tf.metrics.accuracy(labels=labels, predictions=tf.argmax(logits, 1)),
            'loss': tf.metrics.mean(loss)
        }

    # Group the update ops for the tf.metrics
    update_metrics_op = tf.group(*[op for _, op in metrics.values()])

    # Get the op to reset the local variables used in tf.metrics
    metric_variables = tf.get_collection(tf.GraphKeys.LOCAL_VARIABLES, scope="metrics")
    metrics_init_op = tf.variables_initializer(metric_variables)

    # Summaries for training
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    tf.summary.image('train_image', inputs['images'])

    #TODO: if mode == 'eval': ?
    # Add incorrectly labeled images
    mask = tf.not_equal(labels, predictions)

    # Add a different summary to know how they were misclassified
    for label in range(0, params["num_labels"]):
        mask_label = tf.logical_and(mask, tf.equal(predictions, label))
        incorrect_image_label = tf.boolean_mask(inputs['images'], mask_label)
        tf.summary.image('incorrectly_labeled_{}'.format(label), incorrect_image_label)

    # -----------------------------------------------------------
    # MODEL SPECIFICATION
    # Create the model specification and return it
    # It contains nodes or operations in the graph that will be used for training and evaluation
    model_spec = inputs
    model_spec['variable_init_op'] = tf.global_variables_initializer()
    model_spec["predictions"] = predictions
    model_spec['loss'] = loss
    model_spec['accuracy'] = accuracy
    model_spec['metrics_init_op'] = metrics_init_op
    model_spec['metrics'] = metrics
    model_spec['update_metrics'] = update_metrics_op
    model_spec['summary_op'] = tf.summary.merge_all()

    if is_training:
        model_spec['train_op'] = train_op

    return model_spec

In [0]:
def train_sess(sess, model_spec, num_steps, writer, params):
    """Train the model on `num_steps` batches
    Args:
        sess: (tf.Session) current session
        model_spec: (dict) contains the graph operations or nodes needed for training
        num_steps: (int) train for this number of batches
        writer: (tf.summary.FileWriter) writer for summaries
        params: (Params) hyperparameters
    """
    # Get relevant graph operations or nodes needed for training
    loss = model_spec['loss']
    train_op = model_spec['train_op']
    update_metrics = model_spec['update_metrics']
    metrics = model_spec['metrics']
    summary_op = model_spec['summary_op']
    global_step = tf.train.get_global_step()

    # Load the training dataset into the pipeline and initialize the metrics local variables
    sess.run(model_spec['iterator_init_op'])
    sess.run(model_spec['metrics_init_op'])

    # Use tqdm for progress bar
    t = trange(num_steps)
    for i in t:
        # Evaluate summaries for tensorboard only once in a while
        if i % params["save_summary_steps"] == 0:
            # Perform a mini-batch update
            _, _, loss_val, summ, global_step_val = sess.run([train_op, update_metrics, loss,
                                                              summary_op, global_step])
            # Write summaries for tensorboard
            writer.add_summary(summ, global_step_val)
        else:
            _, _, loss_val = sess.run([train_op, update_metrics, loss])
        # Log the loss in the tqdm progress bar
        t.set_postfix(loss='{:05.3f}'.format(loss_val))


    metrics_values = {k: v[0] for k, v in metrics.items()}
    metrics_val = sess.run(metrics_values)
    metrics_string = " ; ".join("{}: {:05.3f}".format(k, v) for k, v in metrics_val.items())
    logging.info("- Train metrics: " + metrics_string)
    
def train_and_evaluate(train_model_spec, eval_model_spec, model_dir, params, restore_from=None):
    """Train the model and evaluate every epoch.
    Args:
        train_model_spec: (dict) contains the graph operations or nodes needed for training
        eval_model_spec: (dict) contains the graph operations or nodes needed for evaluation
        model_dir: (string) directory containing config, weights and log
        params: (Params) contains hyperparameters of the model.
                Must define: num_epochs, train_size, batch_size, eval_size, save_summary_steps
        restore_from: (string) directory or file containing weights to restore the graph
    """
    # Initialize tf.Saver instances to save weights during training
    last_saver = tf.train.Saver() # will keep last 5 epochs
    best_saver = tf.train.Saver(max_to_keep=1)  # only keep 1 best checkpoint (best on eval)
    begin_at_epoch = 0

    with tf.Session() as sess:
        # Initialize model variables
        sess.run(train_model_spec['variable_init_op'])

        # Reload weights from directory if specified
        if restore_from is not None:
            logging.info("Restoring parameters from {}".format(restore_from))
            if os.path.isdir(restore_from):
                restore_from = tf.train.latest_checkpoint(restore_from)
                begin_at_epoch = int(restore_from.split('-')[-1])
            last_saver.restore(sess, restore_from)

        # For tensorboard (takes care of writing summaries to files)
        train_writer = tf.summary.FileWriter(os.path.join(model_dir, 'train_summaries'), sess.graph)
        eval_writer = tf.summary.FileWriter(os.path.join(model_dir, 'eval_summaries'), sess.graph)

        best_eval_acc = 0.0
        for epoch in range(begin_at_epoch, begin_at_epoch + params["num_epochs"]):
            # Run one epoch
            logging.info("Epoch {}/{}".format(epoch + 1, begin_at_epoch + params["num_epochs"]))
            # Compute number of batches in one epoch (one full pass over the training set)
            num_steps = (params["train_size"] + params["train_size"] - 1) // params["batch_size"]
            train_sess(sess, train_model_spec, num_steps, train_writer, params)

            # Save weights
            last_save_path = os.path.join(model_dir, 'last_weights', 'after-epoch')
            last_saver.save(sess, last_save_path, global_step=epoch + 1)

            # Evaluate for one epoch on validation set
            num_steps = (params["eval_size"] + params["batch_size"] - 1) // params["batch_size"]
            metrics = evaluate_sess(sess, eval_model_spec, num_steps, eval_writer)

            # If best_eval, best_save_path
            eval_acc = metrics['accuracy']
            if eval_acc >= best_eval_acc:
                # Store new best accuracy
                best_eval_acc = eval_acc
                # Save weights
                best_save_path = os.path.join(model_dir, 'best_weights', 'after-epoch')
                best_save_path = best_saver.save(sess, best_save_path, global_step=epoch + 1)
                logging.info("- Found new best accuracy, saving in {}".format(best_save_path))
                # Save best eval metrics in a json file in the model directory
                best_json_path = os.path.join(model_dir, "metrics_eval_best_weights.json")
                save_dict_to_json(metrics, best_json_path)

            # Save latest eval metrics in a json file in the model directory
            last_json_path = os.path.join(model_dir, "metrics_eval_last_weights.json")
            save_dict_to_json(metrics, last_json_path)

In [6]:
tf.set_random_seed(101)

params = {
    "learning_rate": 0.001,
    "batch_size": 32,
    "num_epochs": 10,

    "num_channels": 16,
    "use_batch_norm": True,
    "bn_momentum": 0.9,

    "image_size": 64,
    "use_random_flip": True,
    "num_labels": 12,

    "num_parallel_calls": 4,
    "save_summary_steps": 1
}

set_logger(os.path.join(DRIVE_FOLDER, 'train.log'))

# Create the input data pipeline
logging.info("Creating the datasets...")

train_data_dir = os.path.join(DRIVE_FOLDER, "train")
# test_data_dir = os.path.join(DRIVE_FOLDER, "test")
eval_data_dir = os.path.join(DRIVE_FOLDER, "dev")

# Get the filenames from the train and dev sets
train_filenames = [os.path.join(train_data_dir, img_id) for img_id in os.listdir(train_data_dir) if img_id.endswith('.png')]
# test_filenames = [os.path.join(test_data_dir, img_id) for img_id in os.listdir(test_data_dir) if img_id.endswith('.png')]
eval_filenames = [os.path.join(eval_data_dir, img_id) for img_id in os.listdir(eval_data_dir) if img_id.endswith('.png')]

train_labels = [(img_id.split('_')[0]) for img_id in os.listdir(train_data_dir)]
# test_labels = [(img_id.split('_')[0]) for img_id in os.listdir(test_data_dir)]
eval_labels = [(img_id.split('_')[0]) for img_id in os.listdir(eval_data_dir)]
params['train_size'] = len(train_filenames)
# params['test_size'] = len(test_filenames)
params['eval_size'] = len(eval_filenames)

# encode string labels to numerical
le = preprocessing.LabelEncoder()

train_labels = le.fit_transform(train_labels)
eval_labels = le.transform(eval_labels)
print("Labels after encoding:", train_labels)

train_size = len(train_filenames)
eval_size = len(eval_filenames)

print("There are", train_size, "training images and", eval_size, "development images.")

Creating the datasets...


Labels after encoding: [ 3  7  2 ...  9  3 10]
There are 4432 training images and 554 development images.


In [7]:
# Create the two iterators over the two datasets
train_inputs = input_fn(True, train_filenames, train_labels, params)
eval_inputs = input_fn(True, eval_filenames, eval_labels, params)


# Define the model
logging.info("Creating the model...")
train_model_spec = model_fn('train', train_inputs, params)
eval_model_spec = model_fn('eval', eval_inputs, params, reuse=True)

Instructions for updating:
Colocations handled automatically by placer.


From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


From <ipython-input-3-e9bd6209bbc2>:71: DatasetV1.make_initializable_iterator (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.
Creating the model...


Instructions for updating:
Use keras.layers.conv2d instead.


...

From <ipython-input-4-7c9b2304dd88>:33: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use tf.cast instead.


From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/losses/losses_impl.py:209: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


In [8]:
# Train the model
logging.info("Starting training for {} epoch(s)".format(params["num_epochs"]))
train_and_evaluate(train_model_spec, eval_model_spec, DRIVE_FOLDER, params, None)

Starting training for 10 epoch(s)
Epoch 1/10
 16%|█▋        | 45/276 [00:20<01:42,  2.25it/s, loss=1.956]

InvalidArgumentError: ignored